In [1]:
import random
import collections

In [2]:
def pancake_sort(items, debug=False):
    if len(items) <= 1:
        if debug: print('{} -> {}: {}'.format(items, items, []))
        return items, []
    elif len(items) == 2:
        if items[0] < items[1]:
            if debug: print('{} -> {}: {}'.format(items, items, []))
            return items, []
        else:
            if debug: print('{} -> {}: {}'.format(items, list(reversed(items)), [2]))
            return list(reversed(items)), [2]
    else:
        largest = max(items)
        largest_index = items.index(largest)
        flips = []
        if largest_index == len(items) - 1:
            items1 = items
        elif largest_index == 0:
            items1 = list(reversed(items))
            flips = [len(items)]
        else: # largest_index > 0
            items1 = list(reversed(list(reversed(items[:largest_index+1])) + items[largest_index+1:]))
            flips = [largest_index + 1, len(items)]
        if debug: print('{} -> {}: {}'.format(items, items1, flips))
        sorted_items, sorting_flips = pancake_sort(items1[:-1], debug=debug)
        return sorted_items + items1[-1:], flips + sorting_flips

In [3]:
def enflip(items, flips, burnt=False, debug=False):
    if debug: i0 = items
    for flip in flips:
        if burnt:
            items = [-i for i in reversed(items[:flip])] + items[flip:]
        else:
            items = [i for i in reversed(items[:flip])] + items[flip:]
        if debug: print('{} -{}-> {}'.format(i0, flip, items))
        if debug: i0 = items
    return items

def unflip(items, flips, burnt=False, debug=False):
    return enflip(items, reversed(flips), burnt=burnt, debug=debug)

In [4]:
def pancake_adjacent(higher, lower, sorted_items):
    if sorted_items.index(higher) == sorted_items.index(lower) - 1:
        return True
    else:
        return False

In [5]:
def pancake_chunks(items):
    atoms = [[i] for i in items]
    sorted_items = list(sorted(items))
    return coalesce(atoms)

In [6]:
def coalesce(chunks):
    items = sorted(merge_chunks(chunks), key=abs)
    i = 0
    while i < (len(chunks) - 1):
        last_index = items.index(chunks[i][-1])
        next_index = items.index(chunks[i+1][0])
        if chunks[i][-1] > 0 and chunks[i+1][0] > 0 and last_index + 1 == next_index:
            chunks = chunks[:i] + [chunks[i] + chunks[i+1]] + chunks[i+2:]
        elif chunks[i][-1] < 0 and chunks[i+1][0] < 0 and last_index - 1 == next_index:
            chunks = chunks[:i] + [chunks[i] + chunks[i+1]] + chunks[i+2:]
        else:
            i += 1
    return chunks

In [7]:
def chunk_bases(chunks):
    return [c[-1] if c[-1] > 0 else c[0] for c in chunks]

In [8]:
def merge_chunks(chunks):
    return [i for c in chunks for i in c]

In [9]:
def chunk_count_to_item_count(chunks, cpos):
#     print(chunks, cpos, chunks[:cpos])
    return len(merge_chunks(chunks[:cpos]))

In [10]:
def chunk_index(chunks, item):
    """Return the index of the first chunk containing item"""
    return [i for i, c in enumerate(chunks) if item in c][0]

In [11]:
def enflip_chunks(chunks, flips, debug=False):
    if debug: c0 = chunks
    for flip in flips:
        chunks = [[-i for i in reversed(c)] for c in reversed(chunks[:flip])] + chunks[flip:]
        if debug: print('{} ={}=> {}'.format(c0, flip, chunks))
        if debug: c0 = chunks
    return chunks

def unflip_chunks(chunks, flips, debug=False):
    return enflip(chunks, reversed(flips), debug=debug)

In [12]:
def burnt_pancake_step_case1(chunks, all_chunks, items, largest, largest_burntdown, debug=False):
    largest_burntdown_index = chunk_index(chunks, largest_burntdown)
    if largest_burntdown == largest: # case 1(c): largest pancake is facedown, move to bottom of stack
        cflips = [largest_burntdown_index + 1, len(chunks)]
        flips = [items.index(largest_burntdown) + 1, len(merge_chunks(chunks))]
        done_chunks = enflip_chunks(chunks, cflips, debug=debug)
    else:
        largest_burntdown_partner = max(i for i in chunk_bases(chunks) if abs(i) > largest_burntdown)
        largest_burntdown_partner_index = chunk_index(chunks, largest_burntdown_partner)
        if largest_burntdown_partner_index > largest_burntdown_index: # case 1(a): partner is lower than this
            chunks1 = enflip_chunks(all_chunks, [largest_burntdown_partner_index + 1], debug=debug)
            new_lb_pos = chunk_index(chunks1, -largest_burntdown)
            done_chunks = enflip_chunks(chunks1, [new_lb_pos], debug=debug)
            flips = [chunk_count_to_item_count(all_chunks, largest_burntdown_partner_index + 1), 
                     chunk_count_to_item_count(chunks1, new_lb_pos)]
        else:  # case 1(b): partner is higher than this
            chunks1 = enflip_chunks(chunks, [largest_burntdown_index + 1], debug=debug)
            new_lbi_pos = chunk_index(chunks1, -largest_burntdown_partner)
            done_chunks = enflip_chunks(chunks1, [new_lbi_pos], debug=debug)
            flips = [chunk_count_to_item_count(chunks, largest_burntdown_index + 1), 
                     chunk_count_to_item_count(chunks1, new_lbi_pos)]
    return coalesce(done_chunks), flips

In [13]:
def burnt_pancake_step_case2(chunks, all_chunks, debug=False):
    items = merge_chunks(chunks)
    
    if items == list(reversed(sorted(items))):  # invoke -I special case
        if debug: print("2: -I")
        n = len(items)
        flips = [f for fp in [[n, n-1] for _ in range(n)] for f in fp if f != 0]
        done_items = enflip(items, flips, burnt=True, debug=debug)
        done_chunks = pancake_chunks(done_items)
    elif items == sorted(items):  # items are in reverse order, upside down
        if debug: print("2: rev")
        flips = [len(items)]
        done_items = enflip(items, flips, burnt=True, debug=debug)
        done_chunks = pancake_chunks(done_items)
    else:
        candidates = chunk_bases(chunks)
        largest_unsorted = min(candidates)
        next_largest_unsorted = min(i for i in candidates if i > largest_unsorted)
        largest_unsorted_index = chunk_index(chunks, largest_unsorted)
        next_largest_unsorted_index = chunk_index(chunks, next_largest_unsorted)
#         print(largest_unsorted, next_largest_unsorted, largest_unsorted_index, next_largest_unsorted_index)
        while next_largest_unsorted_index > largest_unsorted_index:
            largest_unsorted = next_largest_unsorted
            largest_unsorted_index = next_largest_unsorted_index
            next_largest_unsorted = min(i for i in candidates if i > largest_unsorted)
            next_largest_unsorted_index = chunk_index(chunks, next_largest_unsorted)
        if debug: print("2: general, lu = {}, nlu = {}".format(largest_unsorted, next_largest_unsorted))
        chunks1 = enflip_chunks(chunks, [largest_unsorted_index + 1])
        done_chunks = enflip_chunks(chunks1, [next_largest_unsorted_index], debug=debug)
#         cflips = [largest_unsorted_index + 1, next_largest_unsorted_index]
        flips = [chunk_count_to_item_count(chunks, largest_unsorted_index + 1), 
                     chunk_count_to_item_count(chunks1, next_largest_unsorted_index)]
#         done_chunks = enflip_chunks(chunks, cflips, debug=debug)
    return coalesce(done_chunks), flips

In [14]:
def burnt_pancake_step(chunks0, items, debug=False):
    chunks = chunks0
    largest = max(abs(i) for c in chunks for i in c)
    while chunks[-1][-1] >= largest:
        chunks = chunks[:-1]
        largest = max(abs(i[-1]) for i in chunks)
    largest_burntdown = max(merge_chunks(chunks))
    if debug: print('<<', chunks, chunks0, items, largest, largest_burntdown)
    if largest_burntdown > 0:
        return burnt_pancake_step_case1(chunks, chunks0, items, largest, largest_burntdown, debug=debug)
    else:
        return burnt_pancake_step_case2(chunks, chunks0, debug=debug)

In [15]:
def burnt_pancake_sort(items, fudge_rate=0, debug=False):
    flips = []
    flip_limit = len(items) * 3
    items0 = items
    chunks = pancake_chunks(items)
    while (any(i for i in items if i < 0) or sorted(items) != items) and len(flips) < flip_limit:
        chunks, these_flips = burnt_pancake_step(chunks, items, debug=debug)
        if debug: print('Got chunks:', chunks)
        items = merge_chunks(chunks)
        flips += these_flips
        if random.random() < fudge_rate:
            if debug: c_old = chunks
            its = [abs(i) for i in merge_chunks(chunks)]
            eits = sorted(i for i in items0 if i not in its)
            chunks = coalesce(pancake_chunks(its + eits))
            items = its + eits
            if debug: print('!! Fudge: Converting {} to {}'.format(c_old, chunks))
    return items, flips

In [16]:
def equiv_case(base_unsorted, flips, burnt=False, max_value=10000):
#     new_sample = random.sample(list(range(1, max_value)), k=len(base_unsorted))
    valid = False
    while not valid:
        new_sample = random.sample(list(range(1, max_value)), k=len(base_unsorted))
        valid = len(new_sample) == len(base_unsorted)
    sample = sorted(new_sample)
    return unflip(sample, flips, burnt=burnt)

In [17]:
def burnt_sorted(pancakes):
    return pancakes == sorted(pancakes)

def unburnt_sorted(pancakes):
    simple_pancakes = [abs(p) for p in pancakes]
    return simple_pancakes == sorted(simple_pancakes)

In [18]:
def inverted_count(pancakes):
    return sum(1 for p in pancakes if p < 0)

In [19]:
def cache_flips(start, flips, burnt=False):
    positions = [{'pos': start}]
    stack = start
    for f in flips:
        stack = enflip(stack, [f], burnt=burnt)
        positions += [{'pos': stack, 'move': f}]
    return positions

In [20]:
def show_cached_flips(cache):
    rows = len(cache[0]['pos'])
    middle_row = (rows) // 2
    for r in range(rows):
        for c in cache:
            if r == middle_row and 'move' in c:
                print(' -{}-> '.format(c['move']), end='')
            elif 'move' in c:
                print('      ', end='')
            if c['pos'][r] > 0:
                print('{:2d} '.format(c['pos'][r]), end='')
            else:
                print('{:2d}*'.format(abs(c['pos'][r])), end='')
        print('')

Approach to developing test cases:

1. Find a random pancake stack.
2. Find the burnt pancake sort of that stack: `burnt_flips`
3. Find an equivalent case for those flips: `pancakes`
4. Find a bunch of fudged burnt sorts of the `pancakes`: `fudged`
5. Find a bunch of random fudged pancake sorts: `padding`

To assemble the test case, join together:
* the `burnt_flips`
* some `fudged`
* enough `padding` to make a round number.

In [21]:
ln = 50
n_equivs = 100
fudge_rate = 0.3

start = [i for i in random.sample(list(range(1, ln+1)), k=ln)]
test_flips = {}
_, test_flips['burnt_flips'] = burnt_pancake_sort(start)
test_flips['pancakes'] = equiv_case(start, test_flips['burnt_flips'], burnt=True)
test_flips['fudged'] = [burnt_pancake_sort(start, fudge_rate=fudge_rate)[1] for _ in range(n_equivs)]
test_flips['padding'] = [burnt_pancake_sort(random.sample(list(range(1, ln+1)), k=ln), fudge_rate=fudge_rate)[1] for _ in range(n_equivs)]
len(test_flips)

4

In [22]:
test_data = [test_flips['burnt_flips']]
test_data.extend(random.sample(test_flips['fudged'], k=random.randint(50, 70)))
test_data.extend(random.sample(test_flips['padding'], k=(99-len(test_data))))
len(test_data)

99

In [23]:
random.shuffle(test_data)

In [24]:
sum(1 for f in test_data
   if unburnt_sorted(
       enflip(test_flips['pancakes'], f, burnt=False)))

55

In [25]:
sum(1 for f in test_data
   if burnt_sorted(
       enflip(test_flips['pancakes'], f, burnt=True)))

1

In [26]:
[i+1 for i, f in enumerate(test_data)
   if burnt_sorted(
       enflip(test_flips['pancakes'], f, burnt=True))]

[61]

In [27]:
# random.shuffle(test_data)
# with open('07-flips.txt', 'w') as tdf:
#     tdf.write('burgers: {}\n'.format(' '.join(str(i) for i in test_flips['pancakes'] if i > 0)))
#     for i, c in enumerate(test_data):
#         tdf.write('{:02}: {}\n'.format(i+1, ' '.join(str(i) for i in c if i > 0)))

# Example cases

In [28]:
ln = 6
n_equivs = 5
fudge_rate = 0.7

start = [i for i in random.sample(list(range(1, ln+1)), k=ln)]
test_flips = {}
_, test_flips['burnt_flips'] = burnt_pancake_sort(start)
test_flips['pancakes'] = equiv_case(start, test_flips['burnt_flips'], burnt=True, max_value=ln*2)
test_flips['fudged'] = [burnt_pancake_sort(start, fudge_rate=fudge_rate)[1] for _ in range(n_equivs)]
test_flips['padding'] = [burnt_pancake_sort(random.sample(list(range(1, ln+1)), k=ln), fudge_rate=fudge_rate)[1] for _ in range(n_equivs)]
test_flips

{'burnt_flips': [5, 6, 2, 1, 2, 5, 3, 2, 3, 2, 3, 2],
 'pancakes': [8, 7, 5, 4, 11, 9],
 'fudged': [[5, 6, 1, 5, 4, 3, 4, 3, 4, 3, 4, 3],
  [5, 6, 1, 5, 4, 3, 4, 3, 4, 3, 4, 3],
  [5, 6, 2, 1, 2, 5],
  [5, 6, 1, 5],
  [5, 6, 1, 5, 4, 3, 4, 3, 4, 3, 4, 3]],
 'padding': [[1, 6, 5],
  [5, 6, 1, 4, 1, 2],
  [1, 6, 5, 0, 4, 5, 2, 4, 2, 1, 2, 1],
  [2, 6, 2, 5, 2, 3, 1, 2],
  [2, 6, 4, 0, 1, 4, 1, 3, 1, 2]]}

In [29]:
# test_flips = {'burnt_flips': [4, 5, 2, 1, 2, 3, 1],
#  'fudged': [[4, 5, 2, 1, 2, 3],
#   [4, 5, 1, 3, 2, 1, 2, 1],
#   [4, 5, 1, 3, 2, 1, 2, 1],
#   [4, 5, 1, 3],
#   [4, 5, 2, 1, 2, 3, 1]],
#  'padding': [[2, 5, 1, 2],
#   [2, 5, 2, 1, 3],
#   [1, 3, 1, 2, 1],
#   [2, 5, 4, 1, 2, 3],
#   [4, 5, 3, 4, 3, 1]],
#  'pancakes': [4, 2, 6, 7, 5]}

In [30]:
test_flips = {'burnt_flips': [3, 5, 3, 2, 3, 2],
 'pancakes': [9, 18, 22, 15, 13],
 'fudged': [[3, 5, 2, 3],
  [3, 5, 2, 3],
  [3, 5, 2, 3],
  [3, 5, 2, 3],
  [3, 5, 2, 3]],
 'padding': [[4, 5, 3, 4, 2, 3],
  [3],
  [3, 5, 4, 2, 3, 4, 2],
  [2, 5, 2, 3, 2],
  [3, 5, 3]]}

In [31]:
bf = [f for f in test_flips['burnt_flips'] if f > 0]
bf

[3, 5, 3, 2, 3, 2]

In [32]:
enflip(test_flips['pancakes'], test_flips['burnt_flips'], burnt=True)

[9, 13, 15, 18, 22]

In [33]:
enflip(test_flips['pancakes'], test_flips['fudged'][0], burnt=True)

[-9, -13, -15, 18, 22]

In [34]:
enflip(test_flips['pancakes'], bf, burnt=True)

[9, 13, 15, 18, 22]

In [35]:
show_cached_flips(cache_flips(test_flips['pancakes'], bf))

 9       22       13        9       15       13        9 
18       18       15       15        9        9       13 
22  -3->  9  -5->  9  -3-> 13  -2-> 13  -3-> 15  -2-> 15 
15       15       18       18       18       18       18 
13       13       22       22       22       22       22 


In [36]:
show_cached_flips(cache_flips(test_flips['pancakes'], bf)[:1])

 9 
18 
22 
15 
13 


In [37]:
show_cached_flips(cache_flips(test_flips['pancakes'], bf)[:2])

 9       22 
18       18 
22  -3->  9 
15       15 
13       13 


In [38]:
show_cached_flips(cache_flips(test_flips['pancakes'], bf, burnt=True))

 9       22*      13*       9*      15*      13*       9 
18       18*      15*      15        9        9*      13 
22  -3->  9* -5->  9  -3-> 13  -2-> 13  -3-> 15  -2-> 15 
15       15       18       18       18       18       18 
13       13       22       22       22       22       22 


In [39]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][0], burnt=False)[:1])

 9 
18 
22 
15 
13 


In [40]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][0], burnt=False)[:2])

 9       22 
18       18 
22  -3->  9 
15       15 
13       13 


In [41]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][0], burnt=False))

 9       22       13       15        9 
18       18       15       13       13 
22  -3->  9  -5->  9  -2->  9  -3-> 15 
15       15       18       18       18 
13       13       22       22       22 


In [42]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][0], burnt=True))

 9       22*      13*      15        9*
18       18*      15*      13       13*
22  -3->  9* -5->  9  -2->  9  -3-> 15*
15       15       18       18       18 
13       13       22       22       22 


In [43]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][1], burnt=True))

 9       22*      13*      15        9*
18       18*      15*      13       13*
22  -3->  9* -5->  9  -2->  9  -3-> 15*
15       15       18       18       18 
13       13       22       22       22 


In [44]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][3], burnt=False))

 9       22       13       15        9 
18       18       15       13       13 
22  -3->  9  -5->  9  -2->  9  -3-> 15 
15       15       18       18       18 
13       13       22       22       22 


In [45]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['fudged'][3], burnt=True))

 9       22*      13*      15        9*
18       18*      15*      13       13*
22  -3->  9* -5->  9  -2->  9  -3-> 15*
15       15       18       18       18 
13       13       22       22       22 


In [46]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['padding'][0]))

 9       15       13       18       22       13        9 
18       22        9        9       13       22       22 
22  -4-> 18  -5-> 18  -3-> 13  -4->  9  -2->  9  -3-> 13 
15        9       22       22       18       18       18 
13       13       15       15       15       15       15 


In [47]:
show_cached_flips(cache_flips(test_flips['pancakes'], test_flips['padding'][3]))

 9       18       13       15       22       13 
18        9       15       13       13       22 
22  -2-> 22  -5-> 22  -2-> 22  -3-> 15  -2-> 15 
15       15        9        9        9        9 
13       13       18       18       18       18 


In [48]:
example_data = [test_flips['burnt_flips']]
example_data.extend(random.sample(test_flips['fudged'], k=4))
example_data.extend(random.sample(test_flips['padding'], k=5))
len(example_data)

10

In [49]:
random.shuffle(example_data)
with open('07-example.txt', 'w') as tdf:
    tdf.write('burgers: {}\n'.format(' '.join(str(i) for i in test_flips['pancakes'] if i > 0)))
    for i, c in enumerate(example_data):
        tdf.write('{:02}: {}\n'.format(i+1, ' '.join(str(i) for i in c if i > 0)))